[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/15w47aHUM52MCJViF_LXTuAGFczg29J0g?usp=sharing)

# Práctica 9: RNNs

## Pre-requisitos

### Instalar paquetes

Si la práctica requiere algún paquete de Python, habrá que incluir una celda en la que se instalen. Si usamos un paquete que se ha utilizado en prácticas anteriores, podríamos dar por supuesto que está instalado pero no cuesta nada satisfacer todas las dependencias en la propia práctica para reducir las dependencias entre ellas.

### NOTA: En <font color='red'>Google Colab</font> hay que instalar los paquetes EN CADA EJECUCIÓN

In [ ]:
# Ejemplo de instalación de tensorflow 2.0
#%tensorflow_version 2.x
# !pip3 install tensorflow  # NECESARIO SOLO SI SE EJECUTA EN LOCAL
import tensorflow as tf

# Hacemos los imports que sean necesarios
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import os
import datetime

# Pronóstico de series de tiempo

Vamos a utilizar un [conjunto de datos de series de tiempo meteorológicas](https://www.bgc-jena.mpg.de/wetter/) registradas por el Instituto Max Planck de Biogeoquímica. 

Este conjunto de datos contiene 14 características diferentes, como la temperatura del aire, la presión atmosférica y la humedad. Estos se recopilaron cada 10 minutos, de 2009 a 2016.

Lo primero que vamos a hacer es cargar los datos en un DataFrame de la librería [Pandas](https://pandas.pydata.org/).

En Keras tenemos implementado tanto las capas recurrentes [GRU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU) y [LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM). Sin embargo, es posible crear cualquier capa recurrente que queramos de una manera sencilla. Basta con crear una capa (llamada celda) que indique las operaciones a realizar, y luego encapsularla en la capa [RNN](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RNN). 

Por tanto, en esta clase vamos a tratar de replicar la celda de la capa GRU.

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)
df = pd.read_csv(csv_path)[::6]

## Visualización de los datos

Antes de nada, vamos a echar un vistazo a los datos.

In [ ]:
df.head()

También podemos ver como cambian los datos de las variables a lo largo del tiempo

In [ ]:
plot_cols = ['T (degC)', 'p (mbar)', 'rho (g/m**3)']
plot_features = df[plot_cols]
plot_features.index = df['Date Time']
_ = plot_features.plot(subplots=True)

plot_features = df[plot_cols][:480]
plot_features.index = df['Date Time'][:480]
_ = plot_features.plot(subplots=True)

## Preprocesado de los datos

Primero de nada, vamos a observar las estadísticas de cada dato. Para ello, la librería Pandas nos permite obtener información de una manera muy sencilla.

In [ ]:
df.describe().transpose()

### Limpiar el dataset
En los datos podemos ver cosas extrañas. Por ejemplo, en la velocidad **wv** tenemos, como dato mínimo, valores de -9999m/s, que es una velocidad absurda. En este caso tenemos dos opciones:

1. Eliminar las filas con el error.
1. Poner un dato más fiable.

En nuestro caso, vamos a decidirnos por la última, sustituyendo cualquier valor menor que 0 por un 0.


In [ ]:
## TODO: sustituye los datos negativos de las columnas wv (m/s) y max. wv (m/s) por 0's
wv = df['wv (m/s)']
wv_negativos = None
wv[wv_negativos] = None

wv_max = None
None
None

Si la operación se ha completado satisfactoriamente, puedes volver a evaluar las estadísticas del dataset para comprobar que se han modificado los valores. 

### Transformación de datos

So observamos los datos vemos que tenemos ciertos componentes que necesitan ser preprocesados para evitar errores innecesarios.

#### Viento

El viento se compone de las 3 últimas variables. En especial la última, **wd (deg)**, que marca la dirección del viento. ¿Cuál es el problema que contiene esta variable? Tómate un tiempo para evaluar la problemática.

<font color='red'>TODO:</font> Escribe aquí el motivo por el que crees que la variable **wd (deg)** puede ser problemática en su forma actual.



¿Ya te has dado cuenta del problema? ¡Es hora de arreglarlo! Sustituye el dato por una representación más útil. 

<font color='red'>PISTA:</font> la solución implica dividir la variable en dos nuevos campos.

In [ ]:
# Convertimos los grados a radianes
wd_rad = df.pop('wd (deg)')*np.pi / 180

# TODO: modifica wd (deg) para obtener una mejor representación
# Crea los nuevos componentes e insértalos en el dataframe.
None
None

#### Fecha

La fecha es también algo muy importante en los datos, pero hay que tener en cuenta su periodicidad. En concreto, dos casos:

- Periodicidad en la hora del día. 
- Periodicidad en la época del año.

Un método simple para convertirlo en una señal utilizable es usar el [seno](https://numpy.org/doc/stable/reference/generated/numpy.sin.html) y el [coseno](https://numpy.org/doc/stable/reference/generated/numpy.cos.html) para convertir la hora en señales claras de "Hora del día" y "Hora del año". Hay que operar de tal manera que:

1. $\cos(hora~0) = \cos(hora~24)$ y $\sin(hora~0) = \sin(hora~24)$ en la hora del día.
1. $\cos(1~enero) \approx \cos(31~diciembre)$ y $\sin(1~enero) \approx \sin(31~diciembre)$ en la hora anual.

<font color='red'>PISTA:</font> recuerda que:
- $\cos(0) = \cos(2\pi)$.
- $\sin(0) = \sin(2\pi)$.

In [ ]:
# Vamos a quitar la columna de la fecha
date_time = pd.to_datetime(df.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')

# Lo convertimos en segundos
timestamp_s = date_time.map(pd.Timestamp.timestamp)

# TODO: calcula el número de segundos de un día y de un año
day = None
year = None

# TODO: Extrae, para cada fecha, el seno y el coseno, y añádelos al dataframe
df['Dia sin'] = None
df['Dia cos'] = None
df['Anho sin'] = None
df['Anho cos'] = None

## Partición de datos

Usaremos una división **(70%, 20%, 10%)** para los conjuntos de entrenamiento, validación y prueba. Tenga en cuenta que los datos **no** se mezclan aleatoriamente antes de dividirlos. Esto es por dos razones:

1. Garantiza que aún sea posible dividir los datos en ventanas de muestras consecutivas.
1. Garantiza que los resultados de la validación / prueba sean más realistas y se evalúen en función de los datos recopilados después de que el modelo haya sido entrenado.

In [ ]:
# TODO: parte los datos según el porcentaje estipulado
n_datos = len(df)

train_df = None
val_df = None
test_df = None

# Normalización de los datos

Para que una red neuronal entrene correctamente suele ser necesario que todos los datos tengan una misma escala, o al menos una parecida. Sin embargo, anteriormente hemos visto como algunas variables tienen medias cercanas a 1000, mientras que otras rondan el 0. Por tanto, necesitamos igualarlas.

Existen múltiples maneras de escalar los datos, pero una de las más comunes es la normalización:

$$ \tilde{x} = \dfrac{x - mean(x)}{std(x)} $$

Consiste en restar, **a cada columna**, su media, dividiendo el resultado por su desviación típica. Para realizarlo, podemos hacer uso de las funciones de Pandas [mean](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html) y [std](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.std.html).

Otra cosa a tener en cuenta es que **se usa la media y desviación del conjunto de entrenamiento** para normalizar todos los datos. Esto es así porque es el único conjunto de datos que conoceríamos a priori en un problema real.

In [ ]:
# TODO = normaliza los dataframes
train_mean = None
train_std = None

train_df = None
val_df = None
test_df = None

Ahora podemos observar como los datos están más o menos balanceados, y todos en la misma escala.

In [ ]:
df_std = train_df.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(df.keys(), rotation=90)

## Ventana de datos

Los modelos de esta práctica harán un conjunto de predicciones basadas en una ventana de muestras consecutivas de los datos. La agrupación de los datos se determinará mediante 3 parámetros:

1. **Anchura de la entrada (input_width):** Es el número de datos de diferentes tiempos que vamos a meter en el modelo. 
1. **Anchura de la salida (label_width):** Es el número de datos de diferentes tiempos que vamos a tratar de predecir. 
1. **Retardo de la salida (offset):** Es el tiempo a pasar entre el último dato de la entrada y el primero de la salida.

Aquí podemos ver una serie de distintos ejemplos:

***
`input_width = 6, label_width = 1, offset = 1`

![](https://drive.google.com/uc?export=view&id=1FJ4zNQO1k9mZoZUWgYEZV5QpYx78CuWP)
***

***
`input_width = 24, label_width = 1, offset = 24`

![](https://drive.google.com/uc?export=view&id=1aIRzPiPVVl_OLxYn-6FK0XRi3fkyi6BU)
***

El objetivo de esta parte es **crear una función que divida el dataset de entrada en todos los segmentos posibles de entrada/salida**, teniendo en cuenta los 3 parámetros mencionados anteriormente.

In [ ]:
# TODO: completa la función sliding_window, sustituyendo las partes con None

def sliding_window(data, labels, input_width, label_width=1, offset=1):
    x = []
    y = []

    for i in range(None):
        _x = data[i:None]
        _y = labels[None]
        x.append(_x)
        y.append(_y)

    x, y = np.array(x),np.array(y)

    if len(x.shape) == 2:
        x = x[:,:,np.newaxis]

    if len(y.shape) == 2:
        y = y[:,:,np.newaxis]
    
    return x, y


# Prueba el código
prueba_x, prueba_y = sliding_window(np.arange(5), np.arange(5), input_width=3, label_width=2, offset=2)
print((prueba_x, prueba_y)) # La salida esperada es (array([[[0], [1], [2]], [[1], [2], [3]]]), array([[[2], [3]], [[3], [4]]]))
assert(prueba_x.shape == (2, 3, 1) and prueba_y.shape == (2, 2, 1)) 

Una vez creada la función, se la pasamos a nuestros datos. Primero, definimos los parámetros de la función.

In [ ]:
input_width = 24
label_width = 24
offset = -input_width + 1
target_labels = 'T (degC)' # Vamos a tratar de predecir la temperatura


# TODO: Llama a la función para dividir el dataset
x_train, y_train = None
x_val, y_val = None
x_test, y_test = None



Vamos a comprobar el tamaño de los datos.

In [ ]:
print('x_train shape :', x_train.shape)
print('y_train shape :', y_train.shape)

## Creando el modelo

Una vez obtenido un correcto preprocesado de los datos, vamos a crear el modelo. Nuestro primer modelo constará de dos partes:

1. Capa recurrente ([LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM))
1. Capa densa, para adecuar la salida al tamaño esperado.

Un argumento de constructor importante para todas las capas de keras RNN (como LSTM) es el argumento `return_sequences`. Este parámetro puede configurar la capa de dos formas.

1. Si es `False` (el valor por defecto) la capa solo devuelve el resultado del paso de tiempo final, lo que le da tiempo al modelo para calentar su estado interno antes de hacer una sola predicción:

![](https://drive.google.com/uc?export=view&id=1-pzuZXOF_pratYFLyJRCg_oKZRsKdzbA)

2. Si es `True` la capa devuelve una salida para cada entrada:

![](https://drive.google.com/uc?export=view&id=17SYlF1UqHdCTTxZangKABGEwd_Mse44p)

Esto es útil para:
  * Apilamiento de capas RNN.
  * Entrenamiento de un modelo en múltiples pasos de tiempo simultáneamente.

En nuestro caso, hemos definido nuestros datos con solapamiento, con el objetivo de entrenar nuestro modelo en múltiples pasos.

In [ ]:
# TODO: Completa el modelo con las partes necesarias

lstm_model = tf.keras.models.Sequential([
    None, # Capa recurrente (usa un múmero de unidades de la capa oculta de 32, y establece correctamente el valor de return_sequences).
    None # Capa densa (escoge correctamente su número de unidades)
])

## Entrenando el modelo

Por último, sólo nos queda entrenar el modelo durante 20 iteraciones. Vamos a utilizar un `callback` llamado [`EarlyStopping`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) para que pare el entrenamiento si el resultado en validación no mejora pasadas una serie de iteraciones.

In [ ]:
MAX_EPOCHS = 20
batch_size = 32

def entrenar_modelo(model, train_data, train_label, val_data, val_label, epochs, batch_size, patience=5):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min')

    model.compile(loss=tf.losses.MeanSquaredError(),
                  optimizer=tf.optimizers.Adam(),
                  metrics=[tf.metrics.MeanAbsoluteError()])

    history = model.fit(train_data, train_label, epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(val_data, val_label),
                        callbacks=[early_stopping])
    return history

history = entrenar_modelo(lstm_model, x_train, y_train, x_val, y_val, MAX_EPOCHS, batch_size)


### Visualizando el resultado

Una vez entrenado el modelo, vamos a ver cómo hace la predicción en test, con respecto al resultado esperado.

In [ ]:
def plot_prediction(feed_data, expected_result, model_result, target_label, offset=1, index=0):

    f_data_x = np.arange(feed_data.shape[1])
    e_data_x = np.arange(expected_result.shape[1]) + offset
    plt.plot(f_data_x, feed_data[index])
    plt.plot(e_data_x, expected_result[index], '*r')
    plt.plot(e_data_x, model_result[index], '^g')
    plt.legend(['inputs', 'labels', 'predictions'])
    plt.title(target_label)

# TODO: predecimos el resultado del test con el modelo
model_result = None

target_index = test_df.columns.get_loc(target_labels)

# TODO: llama a la función plot_prediction con los parámetros correctos
plot_prediction(x_test[:, :, target_index], None, None, target_labels, offset, index=0)


La predicción es bastante cercana...pero algo falla. **¡Las temperaturas están mostrando el resultado normalizado!** Revierte la normalización para un correcto visualizado.

In [ ]:
# TODO: llama a la función plot_prediction con los valores sin normalizar
# None

¡Enhorabuena! Has creado tu primera red recursiva. Ahora vamos a hacerlo un poco más complejo.

# Predicción de múltiples pasos a futuro

El modelo que vimos anteriormente utiliza la salida de cada entrada del LSTM para predecir el tiempo siguiente. 

**¿Qué pasaría entonces si uso el mismo modelo para predecir la temperatura en las 24h siguientes?** Si usásemos el mismo modelo, la predicción de la temperatura en el la hora 25 sólo tomaría en cuenta los datos de la temperatura de la primera hora, en lugar de las 24h que tenemos registradas. Por tanto, tendremos que hacer unos cambios en nuestro modelo.

In [ ]:
input_width = 24
label_width = 24
offset = 1
target_labels = 'T (degC)' # Vamos a tratar de predecir la temperatura


# TODO: Llama a la función para dividir el dataset
x_train, y_train = None
x_val, y_val = None
x_test, y_test = None

## Creando el modelo

En este modelo vamos a necesitar acumular la información de las 24h de entrada para empezar a predecir los resultados a la salida. El esquema a seguir es el siguiente:

![](https://drive.google.com/uc?export=view&id=1R1tGHuXo2yueWlNyP1tWfgQLM6YP1xPw)

Por tanto, debemos hacer un cambio en el argumento `return_sequences` que explicamos anteriormente.

A mayores, lo que vamos a hacer es concatenar dos capas LSTM. Para concatenar capas recurrentes, lo que hay que tener en cuenta es que el parámetro `return_sequences=True` tiene que estar activado en todas las capas menos en la última. En dicha capa, el valor de `return_sequences` dependerá de lo que queramos obtener.

In [ ]:
# TODO: Completa el modelo con las partes necesarias

lstm_model = tf.keras.models.Sequential([
    None, # Capa recurrente (usa un múmero de unidades de la capa oculta de 32, y establece correctamente el valor de return_sequences).
    None, # Capa recurrente (usa un múmero de unidades de la capa oculta de 32, y establece correctamente el valor de return_sequences).
    None # Capa densa (escoge correctamente su número de unidades, 
#                      ya que se necesita una unidad por cada característica de salida que queramos predecir, 
#                      y por el número de veces que necesitamos predecir esa característica),
    tf.keras.layers.Reshape((24, 1)) # reorganizamos la salida para que sea del tipo (label_width, n_output_features)
])

## Entrenando el modelo

Por último, sólo nos queda entrenar el modelo durante 20 iteraciones.

In [ ]:
history = entrenar_modelo(lstm_model, x_train, y_train, x_val, y_val, MAX_EPOCHS, batch_size)

### Visualizando el resultado

Una vez entrenado el modelo, vamos a ver cómo hace la predicción en test, con respecto al resultado esperado.

In [ ]:
# TODO: predecimos el resultado del test con el modelo
model_result = None

target_index = test_df.columns.get_loc(target_labels)

# TODO: llama a la función plot_prediction con los parámetros correctos
plot_prediction(None)

Como puedes ver, es muchísimo más difícil predecir el tiempo con mucha antelación.

# ¡ENHORABUENA! Has completado la práctica de Redes recurrentes.

# ¿Deseas saber más?

La redes recursivas también permiten la predicción a posteriori, utilizando como datos de entradas las mismas predicciones de la red en tiempos anteriores. Son las llamadas [redes auto-regresivas](https://eigenfoo.xyz/deep-autoregressive-models/). 

# Trabajo extra

Como trabajo extra, se propone realizar una idea parecida, pero con dificultad añadida.

- Se utilizará una capa [GRU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU) en lugar de la LSTM.
- Se propone predecir todas las variables del dataset al mismo tiempo, en lugar de sólo la temperatura.

In [ ]:
# TODO: escribe el código para el trabajo extra sin ayuda. Usa todos los bloques de código que quieras.